In [171]:
import pandas as pd 
import requests as req
import json
import re

#%pip install shapely
#%pip install geopandas
from shapely.geometry import Point
import geopandas as gpd
#%pip install fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import folium
#%pip install python-Levenshtein

In [173]:
#connect to the EMT de madrid API to obtain my accessToken
url = 'https://openapi.emtmadrid.es/v1/mobilitylabs/user/login/'

headers = {
    'email': '',
    'password': ''
    }

response = req.request('GET', url, headers=headers)
response.json()

{'code': '01',
 'description': 'Token extend  into control-cache Data recovered  OK, (lapsed: 131 millsecs)',
 'datetime': '2024-03-11T18:43:05.719051',
 'data': [{'nameApp': 'OPENAPI MobilityLabs',
   'levelApp': 0,
   'updatedAt': '2024-03-05T18:29:05.2530000',
   'userName': 'veronicagrim@hotmail.com',
   'lastUpdate': {'$date': 1710178787639},
   'idUser': '2c4e1717-b9d6-4bcf-b026-38157bb8f83b',
   'priv': 'U',
   'tokenSecExpiration': 86399,
   'email': 'veronicagrim@hotmail.com',
   'tokenDteExpiration': {'$date': 1710268786639},
   'flagAdvise': True,
   'accessToken': 'f80fe3d1-a76b-452b-bc7e-21302f7228da',
   'apiCounter': {'current': 49,
    'dailyUse': 20000,
    'owner': 0,
    'licenceUse': 'Please mention EMT Madrid MobilityLabs as data source. Thank you and enjoy!',
    'aboutUses': 'If you need to extend the daily use of this API, please, register your App in Mobilitylabs and use your own X-ClientId and  passKey instead of generic login (more info in https://mobilitylab

In [174]:
#get the accessToken
accessToken=response.json()['data'][0]['accessToken']

In [175]:
#connect to the EMT de madrid API to obtain data from all bicimad stations.
url='https://openapi.emtmadrid.es/v1/transport/bicimad/stations/'

headers = {
    'accessToken': accessToken,
    }

response = req.request("GET", url, headers=headers)
estaciones=response.json()

In [176]:
#convert json to data frame
estaciones=estaciones['data']
estaciones_api=pd.DataFrame(estaciones)


In [189]:
#clean name and separate latitude and longitude
names=[f['name'].split('-')[1].strip() for f in estaciones]
coordenadas=[[f['geometry']['coordinates'] for f in estaciones]]
coordenadas

estaciones_api['name']=names
estaciones_api['geometry']=coordenadas[0]  


#separate latitude and longitude
def dividir_lista(row):
    return pd.Series(row['geometry'])
estaciones_api[['longitude', 'latitude']] = estaciones_api.apply(dividir_lista, axis=1)
estaciones_api=estaciones_api.drop(columns=['geometry'])
estaciones_api

,name,number,address,activate,no_available,total_bases,dock_bikes,free_bases,longitude,latitude
0,Metro Callao,2,"Calle Miguel Moya nº 1,",1,0,27,10,17,-3.705690,40.420400
1,Plaza Conde Suchil,3,"Plaza Conde Surchill, 4 ,",1,0,19,4,15,-3.707254,40.430322
2,Fuencarral,5,"Calle Fuencarral nº 106,",1,0,27,24,2,-3.702135,40.428521
3,Colegio Arquitectos,6,"Calle Hortaleza nº 63,",1,0,19,9,10,-3.698447,40.424148
4,Hortaleza,7,"Calle Hortaleza nº 75,",1,0,19,19,0,-3.697771,40.425191
...,...,...,...,...,...,...,...,...,...,...
606,Parque del Retiro,75,Avenida Menéndez pelayo (esquina con calle Poe...,1,0,27,26,1,-3.678475,40.408272
607,"Joaquin Turina, 49",371,"Calle Joaquin Turina, 49,",1,0,27,8,19,-3.761820,40.372458
608,Metro Pueblo Nuevo,482,"Campuzano, 1,",1,0,23,8,15,-3.642339,40.435687
609,Delicias,273,"Calle de las Delicias, 41 ,",1,0,19,1,18,-3.690918,40.403819


In [187]:
#estaciones_api=estaciones_api[['name', 'number', 'address','geometry','activate','no_available','total_bases','dock_bikes','free_bases']]
#estaciones_api

**EMBAJADAS**

In [191]:
#read the csv created from embassies
embajadas = pd.read_csv('../data/processed/embajadas.csv')
embajadas

,Unnamed: 0,title,street-address,latitude,longitude,number,type_of_place,distance
0,0,Consulado de Argentina,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,40.428216,-3.692666,8,consulado,311.900114
1,1,Consulado de Argentina,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,40.428216,-3.692666,10,consulado,319.381126
2,2,Consulado de Argentina,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,40.428216,-3.692666,104,consulado,380.025843
3,3,Consulado de Belice,CALLE TALAVERA 9,40.453376,-3.680978,148,consulado,255.765517
4,4,Consulado de Belice,CALLE TALAVERA 9,40.453376,-3.680978,159,consulado,312.602762
...,...,...,...,...,...,...,...,...
466,466,Embajada de la República de Corea,CALLE GONZALEZ AMIGO 15,40.466107,-3.666833,534,embajada,422.553954
467,467,Embajada de la República de Corea,CALLE GONZALEZ AMIGO 15,40.466107,-3.666833,285,embajada,537.184597
468,468,Embajada del Nepal,CALLE GENERAL ORAA 3 PLANTA bajo izquierda,40.436031,-3.687905,141,embajada,179.924919
469,469,Embajada del Nepal,CALLE GENERAL ORAA 3 PLANTA bajo izquierda,40.436031,-3.687905,142,embajada,291.887270


In [192]:
#add data to the embassy table of the station api table.
result = pd.merge(embajadas, estaciones_api, on='number', how='left')
result

,Unnamed: 0,title,street-address,latitude_x,longitude_x,number,type_of_place,distance,name,address,activate,no_available,total_bases,dock_bikes,free_bases,longitude_y,latitude_y
0,0,Consulado de Argentina,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,40.428216,-3.692666,8,consulado,311.900114,Metro Alonso Martínez,"Plaza de Alonso Martínez nº 6,",1,0,24,8,14,-3.695440,40.427868
1,1,Consulado de Argentina,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,40.428216,-3.692666,10,consulado,319.381126,Marqués de la Ensenada,"Calle Marqués de la Ensenada (junto a edif, Ce...",1,0,23,13,10,-3.692104,40.425403
2,2,Consulado de Argentina,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,40.428216,-3.692666,104,consulado,380.025843,Castellana,"Paseo de la Castellana nº 4,",1,0,23,15,8,-3.689580,40.426770
3,3,Consulado de Belice,CALLE TALAVERA 9,40.453376,-3.680978,148,consulado,255.765517,Serrano 210,"Calle Serrano nº 210,",1,0,23,12,11,-3.681397,40.451117
4,4,Consulado de Belice,CALLE TALAVERA 9,40.453376,-3.680978,159,consulado,312.602762,Paseo de la Habana 63,"Paseo de la Habana nº 63,",1,0,23,3,20,-3.683593,40.454385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,466,Embajada de la República de Corea,CALLE GONZALEZ AMIGO 15,40.466107,-3.666833,534,embajada,422.553954,"Mesena, 102","Calle Mesena, 102 ,",1,0,24,10,14,-3.663406,40.467724
467,467,Embajada de la República de Corea,CALLE GONZALEZ AMIGO 15,40.466107,-3.666833,285,embajada,537.184597,Santa María Magdalena,"Calle Santa María Magdalena, 15 ,",1,0,23,0,23,-3.670600,40.463100
468,468,Embajada del Nepal,CALLE GENERAL ORAA 3 PLANTA bajo izquierda,40.436031,-3.687905,141,embajada,179.924919,Plaza de Emilio Castelar,"Paseo de la Castellana nº 43,",1,0,12,1,11,-3.689392,40.435405
469,469,Embajada del Nepal,CALLE GENERAL ORAA 3 PLANTA bajo izquierda,40.436031,-3.687905,142,embajada,291.887270,Bajo elevado de Eduardo Dato,"Paseo de la Castellana nº 42,",1,0,23,11,12,-3.687915,40.433409


In [193]:
#functions to find the source and destination in the table they are in and return the data you want

def buscar_origen(type_of_place_origen, origen):
    if type_of_place_origen.lower() == 'embajada' or type_of_place_origen.lower() == 'consulado':
        if type_of_place_origen.lower() == 'embajada':
            filtered_results = result[result['type_of_place'] == 'embajada']
        elif type_of_place_origen.lower() == 'consulado':
            filtered_results = result[result['type_of_place'] == 'consulado']
       
        posibles_coincidencias = process.extract(origen, filtered_results['title'], scorer=fuzz.WRatio, limit=3)
    
        ruta_data_origen = filtered_results[filtered_results['title'].isin([x[0] for x in posibles_coincidencias])]
        ruta_data_origen = ruta_data_origen[(ruta_data_origen['no_available'] == 0) & (ruta_data_origen['dock_bikes'] > 0)]

        if len(ruta_data_origen) == 0:
            print(f"No se encontraron datos para el origen '{origen}' que cumplan con las condiciones")
            
        mensaje = f'Hola, estás en la/el {ruta_data_origen["title"].iloc[0]} ({ruta_data_origen["street-address"].iloc[0]}). '
        mensaje += f' La estación de Bicimad más cercana es {ruta_data_origen["name"].iloc[0]}({ruta_data_origen["address"].iloc[0].split(" ,")[0]}) '
        mensaje += f'a {round(ruta_data_origen["distance"].iloc[0])} metros de distancia y con {ruta_data_origen["dock_bikes"].iloc[0]} bicis libres.'
        return mensaje
    
    elif type_of_place_origen.lower() == 'estacion':
        posibles_coincidencias = process.extract(origen, estaciones_api['name'], scorer=fuzz.WRatio, limit=1)

        ruta_data_origen = estaciones_api[estaciones_api['name'].isin([x[0] for x in posibles_coincidencias])]

        if len(ruta_data_origen) == 0:
            print(f"No se encontraron datos para el origen '{origen}' que cumplan con las condiciones")
            
        mensaje = f'Hola, estás en la estación de Bicimad {ruta_data_origen["name"].iloc[0]} ({ruta_data_origen["address"].iloc[0].split(",")[0]}), con {ruta_data_origen["dock_bikes"].iloc[0]} bicis libres.'
        return mensaje
    

def buscar_destino(type_of_place_destino, destino):
    if type_of_place_destino.lower() == 'embajada' or type_of_place_destino.lower() == 'consulado':
        if type_of_place_destino.lower() == 'embajada':
            filtered_results = result[result['type_of_place'] == 'embajada']
        elif type_of_place_destino.lower() == 'consulado':
            filtered_results = result[result['type_of_place'] == 'consulado']
       
        posibles_coincidencias = process.extract(destino, filtered_results['title'], scorer=fuzz.WRatio, limit=3)
    
        ruta_data_destino = filtered_results[filtered_results['title'].isin([x[0] for x in posibles_coincidencias])]
        ruta_data_destino = ruta_data_destino[(ruta_data_destino['no_available'] == 0) & (ruta_data_destino['free_bases'] > 0)]

        if len(ruta_data_destino) == 0:
            print(f"No se encontraron datos para el destino '{destino}' que cumplan con las condiciones")
            
        mensaje = f'Tu destino es la/el {ruta_data_destino["title"].iloc[0]} ({ruta_data_destino["street-address"].iloc[0]}). '
        mensaje += f' Con la estación de Bicimad más cercana es {ruta_data_destino["name"].iloc[0]}({ruta_data_destino["address"].iloc[0].split(" ,")[0]}) '
        mensaje += f'a {round(ruta_data_destino["distance"].iloc[0])} metros de distancia y con {ruta_data_destino["free_bases"].iloc[0]} aparcamientos libres.'
        return mensaje
    
    elif type_of_place_destino.lower() == 'estacion':
        posibles_coincidencias = process.extract(destino, estaciones_api['name'], scorer=fuzz.WRatio, limit=1)

        ruta_data_destino = estaciones_api[estaciones_api['name'].isin([x[0] for x in posibles_coincidencias])]

        if len(ruta_data_destino) == 0:
            print(f"No se encontraron datos para el destino '{destino}' que cumplan con las condiciones")
            
        mensaje = f'Tu destino es la estación de Bicimad {ruta_data_destino["name"].iloc[0]} ({ruta_data_destino["address"].iloc[0].split(",")[0]}), con {ruta_data_destino["free_bases"].iloc[0]} bicis libres.'
        return mensaje

In [194]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair points in degrees 
    # (e.g.: Start Point -> 40.4400607 / -3.6425358 End Point -> 40.4234825 / -3.6292625)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)
coordenadas=[]
def distancia(type_of_place_origen, origen, type_of_place_destino, destino):
    
    lat_start, long_start, lat_finish, long_finish = 0, 0, 0, 0
    
    if type_of_place_origen.lower() == 'embajada' or type_of_place_origen.lower() == 'consulado':
        
        if type_of_place_origen.lower() == 'embajada':
            filtered_results = result[result['type_of_place'] == 'embajada']
        elif type_of_place_origen.lower() == 'consulado':
            filtered_results = result[result['type_of_place'] == 'consulado']
        
        posibles_coincidencias = process.extract(origen, filtered_results['title'], scorer=fuzz.WRatio, limit=3)
    
        ruta_data_origen = filtered_results[filtered_results['title'].isin([x[0] for x in posibles_coincidencias])]
        ruta_data_origen = ruta_data_origen[(ruta_data_origen['no_available'] == 0) & (ruta_data_origen['dock_bikes'] > 0)]
        lat_start = ruta_data_origen['latitude_y'].iloc[0]
        long_start = ruta_data_origen['longitude_y'].iloc[0]
        
    elif type_of_place_origen.lower() == 'estacion':
        posibles_coincidencias = process.extract(origen, estaciones_api['name'], scorer=fuzz.WRatio, limit=1)

        ruta_data_origen = estaciones_api[estaciones_api['name'].isin([x[0] for x in posibles_coincidencias])]
        ruta_data_origen = ruta_data_origen[(ruta_data_origen['no_available'] == 0) & (ruta_data_origen['dock_bikes'] > 0)]

        lat_start = ruta_data_origen['latitude'].iloc[0]
        long_start = ruta_data_origen['longitude'].iloc[0]
        
    if type_of_place_destino.lower() == 'embajada' or type_of_place_destino.lower() == 'consulado':
            
        if type_of_place_destino.lower() == 'embajada':
            filtered_results = result[result['type_of_place'] == 'embajada']
        elif type_of_place_destino.lower() == 'consulado':
            filtered_results = result[result['type_of_place'] == 'consulado']
        
        posibles_coincidencias = process.extract(destino, filtered_results['title'], scorer=fuzz.WRatio, limit=3)
    
        ruta_data_destino = filtered_results[filtered_results['title'].isin([x[0] for x in posibles_coincidencias])]
        ruta_data_destino = ruta_data_destino[(ruta_data_destino['no_available'] == 0) & (ruta_data_destino['free_bases'] > 0)]
        
        lat_finish = ruta_data_destino['latitude_y'].iloc[0]
        long_finish = ruta_data_destino['longitude_y'].iloc[0]
        
        
    elif type_of_place_destino.lower() == 'estacion':
        posibles_coincidencias = process.extract(destino, estaciones_api['name'], scorer=fuzz.WRatio, limit=1)

        ruta_data_destino = estaciones_api[estaciones_api['name'].isin([x[0] for x in posibles_coincidencias])]
        ruta_data_destino = ruta_data_destino[(ruta_data_destino['no_available'] == 0) & (ruta_data_destino['free_bases'] > 0)]

        lat_finish = ruta_data_destino['latitude'].iloc[0]
        long_finish = ruta_data_destino['longitude'].iloc[0]
        
    coordenadas.append((lat_start, long_start, lat_finish, long_finish))
    return f'La distancia total en bici serán {round(distance_meters(lat_start, long_start, lat_finish, long_finish)[0])} metros.'
  

In [195]:
type_of_place_origen= 'consulado'
origen= 'Mali'
type_of_place_destino= 'estacion'
destino= 'legazpi'

print(buscar_origen(type_of_place_origen, origen))
print ( )
print(buscar_destino(type_of_place_destino, destino))
print ( )
print(distancia(type_of_place_origen, origen, type_of_place_destino, destino))

Hola, estás en la/el Consulado de Mali (CALLE ZURBANO 29 PLANTA 2 Izquierda).  La estación de Bicimad más cercana es Santa Engracia(Calle Santa Engracia nº 14,) a 369 metros de distancia y con 11 bicis libres.

Tu destino es la estación de Bicimad Metro Legazpi (Calle Bolívar nº 3), con 15 bicis libres.

La distancia total en bici serán 4247 metros.


In [196]:
lat_center = (coordenadas[0][0] + coordenadas[0][2]) / 2
long_center = (coordenadas[0][1] + coordenadas[0][3]) / 2

start=(coordenadas[0][0], coordenadas[0][1])
finish=(coordenadas[0][2],coordenadas[0][3])

# Crear el mapa centrado en el punto medio de las coordenadas
mapa = folium.Map(location=[lat_center, long_center], zoom_start=14)

# Añadir la línea que conecta los puntos
folium.PolyLine(locations=[start, finish], color='blue').add_to(mapa)

# Mostrar el mapa
mapa